In [1]:
import pandas as pd
import numpy as np
import warnings
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
from tqdm import tqdm
import sys
import os

import importlib

#### packages
import helper.strategy_helper as st
import helper.visualization as vz
import helper.helper as hp
import helper.file_helper as file_hp
import config.config as config

In [2]:
config = config.config()
path = config['PATHS']

derived_path = path['derived_path']
all_tweet_data = path['all_tweet_data']

In [10]:
test_campaign = 'iran_202012'
year = '2020_12'
ops_file_path = os.path.join(all_tweet_data, 
                             year, 
                             test_campaign, 
                            f'{test_campaign}_tweets.pkl.gz')
control_file_path = None

data = st.read_ops_control_data(ops_file_path, 
                             control_file_path, 
                             ['ops'])
df_ops = data['ops']
# df_control = data['control']

In [12]:
df_ops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560571 entries, 0 to 560570
Data columns (total 30 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   tweetid                   560571 non-null  int64  
 1   userid                    560571 non-null  int64  
 2   user_display_name         560571 non-null  object 
 3   user_screen_name          560571 non-null  object 
 4   user_reported_location    417523 non-null  object 
 5   user_profile_description  530518 non-null  object 
 6   user_profile_url          338535 non-null  object 
 7   follower_count            560571 non-null  int64  
 8   following_count           560571 non-null  int64  
 9   account_creation_date     560571 non-null  object 
 10  account_language          560571 non-null  object 
 11  tweet_language            444758 non-null  object 
 12  tweet_text                560571 non-null  object 
 13  tweet_time                560571 non-null  o

In [13]:
df_replies = st.reply_to_external_users(df_ops)

In [21]:
#This is one replying to others

df_only_text = df_replies[['tweetid', 'userid',
                           'tweet_text', 
                           'in_reply_to_userid', 
                           'in_reply_to_tweetid']]

In [52]:
df = (df_only_text
      .groupby(['in_reply_to_userid'])['tweet_text']
      .apply(lambda x: list(set(x)))
      .to_frame('tweet_list')
      .reset_index()
     )

df['count'] = df['tweet_list'].apply(
    lambda x: len(x) 
)

df = df.sort_values(by=['count'],
                    ascending=False
                   )

In [53]:
pd.options.display.max_colwidth = 150 
#3
df.iloc[0]['tweet_list']

['@CNN Russia and #China have ratified a Security Council resolution amending the mechanism for extending #humanitarian aid to Syria through the Turkish border without the permission of the Syrian central government. #Syrian #Syria #saudiwomenslivesmatter',
 '@CNN While the #coronavirus is spreading across the United States, the mortality rate among low-income patients, high-population communities, and blacks is far higher than white patients. #Covid_19 #CoronavirusUSA',
 '@CNN The #Lebanese court will start tomorrow the trial of some former and current Lebanese ministers and officials, whose duties were to manage the port of Beirut. #19thRepresents #VoteForDemocracy',
 "@CNN The current crisis in Palestine and Israel's attempt to annex parts of the West Bank are the result of the Oslo Accords.",
 '@CNN UNICEF has declared a state of emergency for #Yemeni children who are facing starvation as a result of aid shortages which have devastated the country. #COVID__19 #RacistInChief #YemenC